In [33]:
import pandas as pd
import os
import shapely.geometry as sg
data_directory = os.path.join('..', 'data', 'ptv', '20240224')

In [5]:
branch_ids = ['1', '2', '3', '4', '5', '6', '7', '8', '10', '11']
table_names = ['stop_times', 'stops', 'trips', 'routes', 'calendar', 'calendar_dates', 'agency', 'shapes']

In [12]:
def get_df(branch_id, table_name):
    files = [os.path.join(data_directory, f) for f in os.listdir(data_directory) if f.split('-')[1] == str(branch_id) and f.split('-')[2] == table_name]
    if len(files) == 0:
        return None
    return pd.concat([pd.read_csv(f) for f in files])

In [15]:
df = {branch_id: {table_name: get_df(branch_id, table_name) for table_name in table_names} for branch_id in branch_ids}

In [62]:
df_trams = df['3']['stop_times'].groupby('trip_id')['stop_id'].apply(lambda x: x.to_numpy()).to_frame(name='stops_sequence').reset_index()
df_trams = pd.merge(df['3']['trips'], df_trams, on='trip_id')
df_trams = pd.merge(df['3']['routes'], df_trams, on='route_id')
df_trams['stops_count'] = df_trams['stops_sequence'].apply(lambda x: len(x))
df_trams = df_trams.sort_values(by=['route_short_name', 'stops_count'], ascending=False).drop_duplicates(subset=['route_short_name', 'trip_headsign', 'direction_id'], keep='first')

df_tram_stops = pd.merge(df['3']['stop_times'], df['3']['trips'], on='trip_id')
df_tram_stops = pd.merge(df_tram_stops, df['3']['routes'], on='route_id')
df_tram_stops  = df_tram_stops.groupby(['route_short_name', 'trip_headsign', 'direction_id'])['stop_id'].apply(lambda x: x.unique()).to_frame(name='stops').reset_index()

df_trams = df_trams.merge(df_tram_stops, on=['route_short_name', 'trip_headsign', 'direction_id'])

df_trams['stops_count_2'] = df_trams['stops'].apply(lambda x: len(x))
df_trams['stops_set_1'] = df_trams['stops_sequence'].apply(lambda x: sorted(x))
df_trams['stops_set_2'] = df_trams['stops'].apply(lambda x: sorted(x))

In [63]:
df_trams[df_trams['stops_set_1'] != df_trams['stops_set_2']]

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color,service_id,trip_id,shape_id,trip_headsign,direction_id,stops_sequence,stops_count,stops,stops_count_2,stops_set_1,stops_set_2
16,3-67-mjp-5,NaN,67,Melbourne University - Carnegie,0,956C58,FFFFFF,T3,1.T3.3-67-mjp-5.2.H,3-67-mjp-5.2.H,Melbourne University to Carnegie,0,"[19489, 19491, 19492, 19494, 19495, 19497, 194...",57,"[19072, 19073, 19074, 19075, 19076, 19077, 190...",66,"[18411, 18973, 18974, 18975, 18976, 18977, 189...","[18208, 18210, 18211, 18214, 18215, 18217, 182..."
17,3-67-mjp-5,NaN,67,Melbourne University - Carnegie,0,956C58,FFFFFF,T2,100.T2.3-67-mjp-5.5.R,3-67-mjp-5.5.R,Carnegie to Melbourne University,1,"[18411, 18410, 18409, 18408, 18407, 18406, 184...",56,"[18411, 18410, 18409, 18408, 18407, 18406, 184...",65,"[18364, 18365, 18367, 18368, 18369, 18370, 183...","[18199, 18201, 18221, 18222, 18223, 18233, 183..."
28,3-35-mjp-3,NaN,35,Clockwise - Clockwise,0,6B3529,FFFFFF,T0,1.T0.3-35-mjp-3.1.H,3-35-mjp-3.1.H,Clockwise to Clockwise,0,"[18038, 18039, 18052, 18040, 18041, 18042, 180...",30,"[18038, 18039, 18052, 18040, 18041, 18042, 180...",29,"[17880, 18037, 18038, 18038, 18039, 18040, 180...","[17880, 18037, 18038, 18039, 18040, 18041, 180..."
35,3-12-mjp-3,NaN,12,St Kilda (Fitzroy St) - Victoria Gardens,0,007E92,FFFFFF,T0,100.T0.3-12-mjp-3.3.H,3-12-mjp-3.3.H,St Kilda (Fitzroy St) to Victoria Gardens,0,"[4135, 4136, 4137, 4138, 4139, 4140, 4134, 413...",40,"[4135, 4136, 4137, 4138, 4139, 4140, 4134, 413...",42,"[4039, 4084, 4124, 4125, 4126, 4127, 4128, 412...","[4039, 4084, 4124, 4125, 4126, 4127, 4128, 412..."
36,3-12-mjp-3,NaN,12,St Kilda (Fitzroy St) - Victoria Gardens,0,007E92,FFFFFF,T0,129.T0.3-12-mjp-3.6.R,3-12-mjp-3.6.R,Victoria Gardens to St Kilda (Fitzroy St),1,"[19737, 19736, 19735, 21966, 19733, 19732, 197...",39,"[18196, 18197, 4324, 4323, 4331, 4332, 4333, 4...",41,"[4135, 4323, 4324, 4331, 4332, 4333, 4334, 438...","[4135, 4323, 4324, 4331, 4332, 4333, 4334, 438..."
